In [2]:
import numpy as np
import os
from time import perf_counter

import pandas as pd

from helpers import *

# Log

## Description of the problem

We are interested in the problem involving complete $0/1$ matrices. Such a matrix is a $m \times 2^m -1$, where the columns are all the distinct binary strings on $m$ bits. Given an integer vector $b$ we want to determine whether there exists a $(2^m -1)$- bit binary vector $\mathbf{x}$ such that $\mathbf{Ax}=\mathbf{b}$.

The problem can be divided into two parts: feasibility and solution. With feasibility, we want to explore whether given an integer vector $b$ i a solution exists (yes or no), and with the latter we wish to investigate the properties needed for solving the problem (i.e.: outputting the vector $x$
.

We wish to explore properties of the problem (or in particular the vector $b$) to determine partial solutions to the problem to reduce the general problem to simpler problems.



## Week 3

It is easy to find a solution for a rhs of the form $\mathbf{b} = [b \quad ... \quad b]$. Since each vector has a complement, we can simply choose $b$ vector complement pairs to build the solution. As a result we choose to simplify the problem and decompose it. For each potential right-hand side, we can find an $l \geq 0$ such that $\mathbf{b} = l \cdot \mathbf{1} + \mathbf{u}$.

We need to have that $\mathbf{u}$ is minimal feasible solution with regards to the $l_{\infty}$ norm, which implies that $\mathbf{u} - \mathbf{1}$ should not be feasible.

We will therefore try to enumerate the set $U$ of vectors that are feasible and minimal with regards to the $l_{\infty}$ norm. We will also try to find their minimal support solutions, and find if there is a case where more than one solution is possible.

To find such vectors, we simply choose vectors one by one, and automatically remove their complement from consideration

In [26]:
m = 16
dt = bool

time_start = perf_counter()
complete_mat = make_complete(m, dt=dt)
time_stop = perf_counter()

df = pd.DataFrame(complete_mat.astype(int))

print("m =", m, "\t Datatype =", dt)
print("Time taken =", 1000*(time_stop-time_start), "ms")

# print(df.to_string(header=False, index=False))

m = 16 	 Datatype = <class 'bool'>
Time taken = 8.375918001547689 ms


It is useful to state some facts about the solution. Given a $\mathbf{u}$ that is minimal feasible, we have that $\mathbf{Ax} = l \cdot \mathbf{1} + \mathbf{u}$. Then suppose that $\mathbf{A\hat{x}} = l \cdot \mathbf{1}$ and $\mathbf{A\tilde{x}} = \mathbf{u}$. Then any vector from $A$ that is chosen to create $\mathbf{u}$ cannot be a vector chosen to create $l \cdot \mathbf{1}$.

Upon choosing columns of $\mathbf{A}$ to enumerate the set $U$ we need to remember that this imposes a constraint on the number of complement pairs columns that can be chosen. Suppose $\mathbf{u}$ is made of $k$ vectors. Since none of these columns sum up to $\mathbf{1}$, in particular it doesn't contain any vector complement pairs.

Suppose we have the given decomposition $\mathbf{Ax} = k \cdot \mathbf{1} + \mathbf{u}$. Then it must hold that at least $2k$ vectors are already chosen, in order to yield the $k \cdot \mathbf{1}$ vector. There are $\binom{2^{m-1}}{k}$ ways of choosing those vectors, if only pairs are considered.

TODO:

 - Investigate properties of vectors that add up to $\mathbf{1}$
 - Given an integer $m$ find the number of minimal feasible vectors $\mathbf{u}$ that can be created, and for each, one determine how many non minimal vectors are feasible
    - Derive an enumeration technique

## Greedy algorithm for finding a solution

In [27]:
def find_solution(b):
    m = b.shape[0]
    dt = bool
    mat = make_complete(m, dt)


In [33]:
b = np.array([1, 2, 3, 4, 5, 6])
sol = find_solution(b)

LinAlgError: Last 2 dimensions of the array must be square

Goals for Week 4:

- Find a counter example for the uniqueness of the support minimal solution, if it's possible.
- If not try to derive a proof ro prove uniqueness or non-uniqueness of the support minimal solution.
- If the solution is not unique, is there a particular property that relates them in a certain way, or are they seemingly random.
- Think about an interpretation for what it means to have a unique solution in the polytope view.

- Have a look at the proof 